In [40]:
# MANIPULACIÓN Y ANÁLISIS DE DATOS

import numpy as np
import pandas as pd

# MODELADO DE MACHINE LEARNING

from sklearn.model_selection import train_test_split,cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import balanced_accuracy_score, accuracy_score, classification_report, silhouette_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

# GUARDADO Y CARGADO DE MODELOS ENTRENADOS
import pickle
from pathlib import Path

In [46]:
cleaned_aq_numerico = pd.read_csv("../data/cleaned_file/air_quality.csv")

In [18]:
def niveles_NO2(concentracion_NO2):
   if concentracion_NO2 <= 80:
    return "Baja"
   elif concentracion_NO2 <= 160:
    return "Moderada"
   else:
    return "Alta"

cleaned_aq_numerico["NO2_Concentración"]= cleaned_aq_numerico["NO2(GT)"].apply(niveles_NO2)

In [19]:
# División de datos

X= cleaned_aq_numerico.drop(["NO2(GT)", "NO2_Concentración"],axis=1)
Y= cleaned_aq_numerico["NO2_Concentración"]

X_train, X_test, Y_train, Y_test =train_test_split(X,Y,test_size=0.2,random_state=42)

In [20]:
# Estandarización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
clf = RandomForestClassifier(random_state=40)

cross_validation_scores = cross_val_score(clf, X_train, Y_train, cv=5, scoring='balanced_accuracy')

print("Precisión promedio en validación cruzada: {:.2f}".format(cross_validation_scores.mean()))

Precisión promedio en validación cruzada: 0.87


In [24]:
aq_regression = LogisticRegression()
aq_regression.fit(X, Y)

prediccion_RL = aq_regression.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_RL)
print("Precisión del modelo de Regresión Logística:", accuracy)

Precisión del modelo de Regresión Logística: 0.8482905982905983


In [44]:
aq_svm = SVC()

aq_svm.fit(X, Y)

prediccion_SVC = aq_svm.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_SVC)
print("Precisión del modelo de SVC:", accuracy)

Precisión del modelo de SVC: 0.8611111111111112


In [26]:
aq_knn = KNeighborsClassifier()

aq_knn.fit(X, Y)

prediccion_KNN = aq_knn.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_KNN)
print("Precisión del modelo de KNN:", accuracy)

Precisión del modelo de KNN: 0.9220085470085471


In [27]:
aq_tree = DecisionTreeClassifier()

aq_tree.fit(X, Y)

prediccion_AD = aq_tree.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_AD)
print("Precisión del modelo de Árbol de Decisión:", accuracy)

Precisión del modelo de Árbol de Decisión: 1.0


In [28]:
aq_forest = RandomForestClassifier()

aq_forest.fit(X, Y)

prediccion_RF = aq_forest.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_RF)
print("Precisión del modelo de Random Forest:", accuracy)

Precisión del modelo de Random Forest: 1.0


In [29]:
aq_adaboost = AdaBoostClassifier()

aq_adaboost.fit(X, Y)
prediccion_ADABOOST = aq_adaboost.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_ADABOOST)
print("Precisión del modelo de AdaBoost:", accuracy)

Precisión del modelo de AdaBoost: 0.8082264957264957


In [30]:
aq_bagging = BaggingClassifier(DecisionTreeClassifier(), n_estimators=300,
                            max_samples=10, n_jobs=-1, random_state=42)
aq_bagging.fit(X, Y)

prediccion_Bagging = aq_bagging.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_Bagging)
print("Precisión del modelo de Bagging:", accuracy)

Precisión del modelo de Bagging: 0.8173076923076923


In [31]:
aq_mlp = MLPClassifier(hidden_layer_sizes=(64, 32, 16), activation='relu', alpha=0.001)

aq_mlp.fit(X, Y)
prediccion_MLP = aq_mlp.predict(X_test)

accuracy = accuracy_score(Y_test, prediccion_MLP)
print("Precisión del modelo de MLP:", accuracy)

Precisión del modelo de MLP: 0.8530982905982906


In [ ]:
modelos = {
    "Regresión Logística": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Arbol de Decisión": DecisionTreeClassifier(),
    "SVC": SVC(),
    "KNN": KNeighborsClassifier(),
    "Bagging": BaggingClassifier(DecisionTreeClassifier())
}

hiperparametros = {
    "Regresión Logística": {'modelo__C': [1, 5, 10], 'modelo__penalty': ['l1','l2']},
    "Random Forest": {'modelo__n_estimators': [75, 100, 125], 'modelo__max_depth': [None, 5, 10, 15, 20], 'modelo__min_samples_split': [1, 5, 10], 'modelo__min_samples_leaf': [1, 5, 10]},
    "AdaBoost": {'modelo__n_estimators': [50, 100, 150], 'modelo__learning_rate': [0.1, 1, 10]},
    "Arbol de Decisión": {'modelo__criterion': ['gini','entropy'], 'modelo__splitter': ['best','random'], 'modelo__max_features': [None,'sqrt']},
    "SVC": {'modelo__C': [0.01, 0.1, 1, 10], 'modelo__kernel': ['rbf', 'sigmoid'], 'modelo__gamma': ['scale','auto']},
    "KNN": {'modelo__n_neighbors': [1, 5, 10], 'modelo__weights': ['uniform','distance'], 'modelo__algorithm': ['ball_tree', 'kd_tree', 'brute', 'auto']},
    "Bagging": {'modelo__n_estimators':[100, 200, 300], 'modelo__max_samples':[5,10,15], 'modelo__n_jobs':[-1, 0, 1]}
    }

pipeline = []

for nombre_modelo, modelo in modelos.items():
    print(f"Entrenando y evaluando el modelo {nombre_modelo}")

    # Obtengo hiperparámetros específicos para el modelo
    hiperparametros_modelo = hiperparametros.get(nombre_modelo, {})

    pipeline = Pipeline([('modelo', modelo)])

    # Verifico si el modelo admite búsqueda de cuadrícula
    if hiperparametros_modelo:
        grid_search = GridSearchCV(estimator=pipeline, param_grid=hiperparametros_modelo, scoring='accuracy', cv=3)
        grid_search.fit(X, Y)
        mejor_modelo = grid_search.best_estimator_
    else:
        mejor_modelo = pipeline.fit(X, Y)

    prediccion_entrenamiento = mejor_modelo.predict(X_train)
    prediccion_prueba = mejor_modelo.predict(X_test)

    reporte_entrenamiento = classification_report(Y_train, prediccion_entrenamiento)
    reporte_prueba = classification_report(Y_test, prediccion_prueba)

    print(f"Informe de Clasificación para {nombre_modelo} en datos de entrenamiento:")
    print(reporte_entrenamiento)
    print(f"Informe de Clasificación para {nombre_modelo} en datos de prueba:")
    print(reporte_prueba)

Entrenando y evaluando el modelo Regresión Logística
Informe de Clasificación para Regresión Logística en datos de entrenamiento:
              precision    recall  f1-score   support

        Alta       0.75      0.60      0.67      1023
        Baja       0.91      0.90      0.91      2966
    Moderada       0.81      0.86      0.84      3496

    accuracy                           0.84      7485
   macro avg       0.82      0.79      0.80      7485
weighted avg       0.84      0.84      0.84      7485

Informe de Clasificación para Regresión Logística en datos de prueba:
              precision    recall  f1-score   support

        Alta       0.76      0.62      0.68       238
        Baja       0.91      0.89      0.90       729
    Moderada       0.82      0.88      0.85       905

    accuracy                           0.85      1872
   macro avg       0.83      0.79      0.81      1872
weighted avg       0.85      0.85      0.85      1872

Entrenando y evaluando el modelo Rando

In [ ]:
report = classification_report(Y_test, prediccion_MLP)
print("\nInforme de clasificación:\n", report)


Informe de clasificación:
               precision    recall  f1-score   support

        Alta       0.85      0.57      0.68       238
        Baja       0.99      0.73      0.84       729
    Moderada       0.75      0.97      0.84       905

    accuracy                           0.83      1872
   macro avg       0.86      0.76      0.79      1872
weighted avg       0.85      0.83      0.82      1872



In [ ]:
modelos = {
    "Regresión Logística": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Arbol de Decisión": DecisionTreeClassifier(),
    "SVC": SVC(),
    "KNN": KNeighborsClassifier(),
    "Bagging": BaggingClassifier(DecisionTreeClassifier()),
    "MLP": MLPClassifier()
    }

metricas = ["accuracy", "f1_macro", "recall_macro", "precision_macro", "roc_auc_ovr"]

resultados_dict = {}

for nombre_modelo, modelo in modelos.items():
    cv_resultados = cross_validate(modelo, X, Y, cv=5, scoring=metricas)
    
    for metrica in metricas:
        
        clave = f"{nombre_modelo}_{metrica}"
        resultados_dict[clave] = cv_resultados[f"test_{metrica}"].mean()
    
resultados = pd.DataFrame([resultados_dict])

In [ ]:
resultados.T.sort_values(by=0, ascending=False)

,0
Regresión Logística_roc_auc_ovr,0.921666
Random Forest_roc_auc_ovr,0.901486
MLP_roc_auc_ovr,0.884915
Bagging_roc_auc_ovr,0.880935
KNN_roc_auc_ovr,0.830618
SVC_precision_macro,0.819619
SVC_accuracy,0.804740
Regresión Logística_precision_macro,0.800427
Regresión Logística_accuracy,0.779525
Random Forest_accuracy,0.779298


In [32]:
models_gridsearch = {}

models = [
    ("Regresión Logística", LogisticRegression()),
    ("Random Forest", RandomForestClassifier()),
    ("AdaBoost", AdaBoostClassifier()),
    ("Arbol de Decisión", DecisionTreeClassifier()),
    ("SVC", SVC()),
    ("KNN", KNeighborsClassifier()),
    ("MLP", MLPClassifier())
]

for model_name, model_instance in models:
    models_gridsearch[model_name] = GridSearchCV(model_instance,
                                                 param_grid={},
                                                 cv=10,
                                                 scoring="accuracy",
                                                 verbose=1,
                                                 n_jobs=-1)
    
    models_gridsearch[model_name].fit(X, Y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [33]:
best_grids = [(i, j.best_score_) for i, j in models_gridsearch.items()]

best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)
best_grids

,Grid,Best score
1,Random Forest,0.827395
4,SVC,0.821833
0,Regresión Logística,0.821302
3,Arbol de Decisión,0.780587
2,AdaBoost,0.765524
5,KNN,0.759851
6,MLP,0.699684


In [34]:
best_model_name = None
best_model_params = None
best_model_score = float('-inf')

for model_name, grid_search_result in models_gridsearch.items():
    if grid_search_result.best_score_ > best_model_score:
        best_model_name = model_name
        best_model_params = grid_search_result.best_params_
        best_model_score = grid_search_result.best_score_

print("El mejor modelo es:", best_model_name)
print("Mejores hiperparámetros:", best_model_params)
print("Puntuación del mejor modelo:", best_model_score)

El mejor modelo es: Random Forest
Mejores hiperparámetros: {}
Puntuación del mejor modelo: 0.8273948763654646


Random Forest está obteniendo una puntuación alta en accuracy debido a su capacidad para ajustarse bien a los datos de entrenamiento, pero esto no garantiza que sea el mejor modelo en términos de precision y recall en un conjunto de datos de prueba.

In [35]:
resultados = pd.DataFrame({
    'Modelo': ['Regresión Logística', 'Random Forest', 'MLP', 'Bagging', 'KNN', 'SVC', 'Arbol de Decisión', 'AdaBoost'],
    'roc_auc_ovr': [0.921666, 0.901486, 0.884915, 0.880935, 0.830618, None, 0.779259, 0.740247],
    'precision_macro': [0.800427, 0.740452, 0.757216, 0.743867, 0.697784, 0.819619, 0.688160, 0.670092],
    'recall_macro': [0.723294, 0.729068, 0.666442, 0.728911, 0.674796, 0.735603, 0.697577, 0.662561],
    'f1_macro': [0.720221, 0.708230, 0.662757, 0.707485, 0.644973, None, 0.671422, 0.631116],
    'accuracy': [0.779525, 0.779298, 0.734850, 0.765729, 0.710895, 0.804740, 0.738050, 0.672539]
})

mejor_modelo_precision = resultados.loc[resultados['precision_macro'].idxmax(), ['Modelo', 'precision_macro']]
mejor_modelo_recall = resultados.loc[resultados['recall_macro'].idxmax(), ['Modelo', 'recall_macro']]
mejor_modelo_f1score = resultados.loc[resultados['f1_macro'].idxmax(), ['Modelo', 'f1_macro']]
mejor_modelo_roc_auc = resultados.loc[resultados['roc_auc_ovr'].idxmax(), ['Modelo', 'roc_auc_ovr']]
mejor_modelo_accuracy = resultados.loc[resultados['accuracy'].idxmax(), ['Modelo', 'accuracy']]

print("Mejor modelo para precision:")
print(mejor_modelo_precision)
print("\nMejor modelo para recall:")
print(mejor_modelo_recall)
print("\nMejor modelo para F1 Score:")
print(mejor_modelo_f1score)
print("\nMejor modelo para ROC AUC:")
print(mejor_modelo_roc_auc)
print("\nMejor modelo para accuracy:")
print(mejor_modelo_accuracy)

Mejor modelo para precision:
Modelo                  SVC
precision_macro    0.819619
Name: 5, dtype: object

Mejor modelo para recall:
Modelo               SVC
recall_macro    0.735603
Name: 5, dtype: object

Mejor modelo para F1 Score:
Modelo      Regresión Logística
f1_macro               0.720221
Name: 0, dtype: object

Mejor modelo para ROC AUC:
Modelo         Regresión Logística
roc_auc_ovr               0.921666
Name: 0, dtype: object

Mejor modelo para accuracy:
Modelo          SVC
accuracy    0.80474
Name: 5, dtype: object


Como se observa, según los valores de las métricas difiere el modelo que resulta optimo. ¿Cuál elegir entonces?

#### Modelo seleccionado: SVC

De acuerdo al contexto del problema, por un lado, se podría deducir que resulta de gran importancia tener en cuenta la precisión. Esto debido a que podría ser relevante minimizar los falsos positivos, es decir, predecir incorrectamente que la concentración de NO2 es alta cuando en realidad es baja. Esto es importante porque podría llevar a acciones incorrectas de mitigación o alerta que no son necesarias, lo que podría generar preocupación innecesaria para el público.  

A su vez, maximizar el recall podría ser importante para garantizar que las concentraciones de NO2 que realmente son altas sean identificadas correctamente, ya que una subestimación de las concentraciones de NO2 podría llevar a una falta de acciones de mitigación necesarias.

In [37]:
models_gridsearch['SVC'].best_estimator_

SVC()

In [38]:
models_gridsearch['SVC'].best_estimator_.score(X_test, Y_test)

0.8611111111111112

In [47]:
svc_model = Path().cwd().parent.parent / "model" / "svc.model.pkl"

with open(svc_model, "wb") as archivo_salida:
    pickle.dump(models_gridsearch['SVC'].best_estimator_, archivo_salida)